In [1]:
from recup_dataset import *
from caracteristiques_v4 import *
#from lieux_4 import *
from lieux_5 import *
from vehicules_3 import *
from usagers_3 import *
from merge_file import *

In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

## <span style="color:blue"> Import des données d'entrainement' </span>

In [3]:
dfs = recuperer_train()

df_c, df_l, df_v, df_u = merge_c_v_l_u(dfs)

df_carac = traitement_caracteristiques(df_c)
df_li = traitement_lieux(df_l)
df_ve = traitement_vehicules_2(df_v)
df_u = traitement_usagers_2(df_u)

C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\recup_dataset.py:36: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin_complet, on_bad_lines='skip', encoding=encodage, sep =';')
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\recup_dataset.py:36: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin_complet, on_bad_lines='skip', encoding=encodage, sep =';')
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\recup_dataset.py:36: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin_complet, on_bad_lines='skip', encoding=encodage, sep =';')
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\re

C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\lieux_5.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['plan'] = df_l['plan'].apply(plan_2)
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\lieux_5.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['prof'] = df_l['prof'].fillna(-1)
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\lieux_5.py:153: Settin

## <span style="color:green"> On merge tous les datasets entre eux sur Num_Acc en train </span>

In [6]:
df_final = tout_merge(df_u, df_ve, df_li, df_carac)

In [7]:
# age moyen par accident 
def annee(value):
    return int("20" + str(int(value)))

df_final['an'] = df_final['an'].apply(annee)
df_final['an']

df_final['age_moy_acc'] = df_final['an'] - df_final['moyenne_an_nais']
df_final = df_final.drop(columns=['moyenne_an_nais'])

In [8]:
def vitesse_1(value):
    if value > 130 :
        return -1
    else :
        return value 
    
def vitesse_2(value):
    if value <= 30:
        return 30
    elif value <= 50:
        return 50
    elif value <= 70:
        return 70
    elif value <= 90:
        return 90
    elif value <= 110:
        return 110
    elif value <= 130:
        return 130
    else :
        return -1

def vitesse_3(value): # par ordre de dangereusité
    if value == 30 : # en ville
        return 1
    elif value in [50, 110]: # agglomération ou vitesse "lente" sur autoroute
        return 2
    elif value == 70: # vitesse "lente" en campagne
        return 3
    elif value == 130: # ahute vitesse autoroute
        return 4
    elif value == 90: # haute vitesse en campagne
        return 5

In [9]:
df_final['vma'] = df_final['vma'].apply(vitesse_1)
# Remplacer les NaN dans la colonne 'vma' par 30 si 'agg' vaut 1 et par 50 si 'agg' vaut 2
df_final['vma'] = df_final.apply(lambda row: 80 if pd.isna(row['vma']) and row['agg'] == 1 else row['vma'], axis=1)
df_final['vma'] = df_final.apply(lambda row: 50 if pd.isna(row['vma']) and row['agg'] == 2 else row['vma'], axis=1)


# Remplacer les valeurs de '-1' dans la colonne 'vma' par 80 si 'agg' vaut 1 et par 50 si 'agg' vaut 2
df_final['vma'] = df_final.apply(lambda row: 80 if row['vma'] == -1 and row['agg'] == 1 else row['vma'], axis=1)
df_final['vma'] = df_final.apply(lambda row: 50 if row['vma'] == -1 and row['agg'] == 2 else row['vma'], axis=1)

df_final['vma'] = df_final['vma'].apply(vitesse_2)
df_final['vma'] = df_final['vma'].apply(vitesse_3)

In [24]:
df_ridge = df_final.drop(columns=['etatp_sans_2','nb_place_arriere','nb_place_avant','surf','vosp'])

## <span style="color:blue"> Import des données de test </span>

In [15]:
# chemin vers le csv
directory = "TEST/TEST/"

# Importez chaque fichier CSV
df_caracteristiques_test = pd.read_csv(directory + "CARACTERISTIQUES.csv")
df_lieux_test = pd.read_csv(directory + "LIEUX.csv")
df_usagers_test = pd.read_csv(directory + "USAGERS.csv")
df_vehicules_test = pd.read_csv(directory + "VEHICULES.csv")

ve_test = traitement_vehicules_2(df_vehicules_test)
c_test = traitement_caracteristiques(df_caracteristiques_test)
l_test = traitement_lieux(df_lieux_test)
u_test = traitement_usagers_2(df_usagers_test,True)

C:\Users\vince\AppData\Local\Temp\ipykernel_16264\237960089.py:5: DtypeWarning: Columns (4,10,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_caracteristiques_test = pd.read_csv(directory + "CARACTERISTIQUES.csv")
C:\Users\vince\AppData\Local\Temp\ipykernel_16264\237960089.py:6: DtypeWarning: Columns (7,8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lieux_test = pd.read_csv(directory + "LIEUX.csv")
C:\Users\vince\AppData\Local\Temp\ipykernel_16264\237960089.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_usagers_test = pd.read_csv(directory + "USAGERS.csv")
C:\Users\vince\AppData\Local\Temp\ipykernel_16264\237960089.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicules_test = pd.read_csv(directory + "VEHICULES.csv")
C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Ap

C:\Users\vince\OneDrive\Documents\ESIEE_Paris\E4\Semestre_2\Apprentissage_automatique_2\projet\lieux_5.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_l['catr'] = df_l['catr'].apply(catr_2)


## <span style="color:green"> On merge tous les datasets entre eux sur Num_Acc pour le test </span>

In [16]:
df_test = tout_merge(u_test, ve_test, l_test, c_test)

In [17]:
# calcul de l'age moyen des accidentés par accident
df_test['an'] = df_test['an'].apply(annee)
df_test['an']

df_test['age_moy_acc'] = df_test['an'] - df_test['moyenne_an_nais']
df_test = df_test.drop(columns=['moyenne_an_nais'])

In [18]:
df_test['vma'] = df_test['vma'].apply(vitesse_1)
# Remplacer les NaN dans la colonne 'vma' par 80 si 'agg' vaut 1 et par 50 si 'agg' vaut 2
df_test['vma'] = df_test.apply(lambda row: 80 if pd.isna(row['vma']) and row['agg'] == 1 else row['vma'], axis=1)
df_test['vma'] = df_test.apply(lambda row: 50 if pd.isna(row['vma']) and row['agg'] == 2 else row['vma'], axis=1)


# Remplacer les valeurs de '-1' dans la colonne 'vma' par 80 si 'agg' vaut 1 et par 50 si 'agg' vaut 2
df_test['vma'] = df_test.apply(lambda row: 80 if row['vma'] == -1 and row['agg'] == 1 else row['vma'], axis=1)
df_test['vma'] = df_test.apply(lambda row: 50 if row['vma'] == -1 and row['agg'] == 2 else row['vma'], axis=1)

df_test['vma'] = df_test['vma'].apply(vitesse_2)
df_test['vma'] = df_test['vma'].apply(vitesse_3)

In [19]:
Num_Acc_test = df_test['Num_Acc']
test_officiel = df_test.drop(columns=['Num_Acc'])

In [20]:
test_officiel = test_officiel.drop(columns=['etatp_sans_2','nb_place_arriere','nb_place_avant','surf','vosp'])

## <span style="color:red"> Jeu d'entrainement </span>

In [28]:
X = df_ridge.drop(columns=['grave', 'Num_Acc'])
y = df_ridge['grave']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### <span style="color:orange"> CatBoostClassifer  (modèle 1)</span>

In [30]:
from catboost import CatBoostClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

params_cat = {'depth': 10, 'iterations': 1850, 'l2_leaf_reg': 7, 'learning_rate': 0.0325}

# parametres du modèle
catboost_model = CatBoostClassifier(**params_cat)

# Entraînement
catboost_model.fit(X_train, y_train)

# prédictions sur jeu de test
y_pred_proba = catboost_model.predict_proba(X_test)[:, 1]

# Calcule de l'AUC
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC Score avec CatBoost:", auc)


0:	learn: 0.6826228	total: 73.6ms	remaining: 2m 16s
1:	learn: 0.6722128	total: 145ms	remaining: 2m 13s
2:	learn: 0.6625923	total: 213ms	remaining: 2m 11s
3:	learn: 0.6536300	total: 283ms	remaining: 2m 10s
4:	learn: 0.6455832	total: 355ms	remaining: 2m 10s
5:	learn: 0.6379329	total: 424ms	remaining: 2m 10s
6:	learn: 0.6307800	total: 491ms	remaining: 2m 9s
7:	learn: 0.6242668	total: 565ms	remaining: 2m 10s
8:	learn: 0.6183424	total: 633ms	remaining: 2m 9s
9:	learn: 0.6125395	total: 702ms	remaining: 2m 9s
10:	learn: 0.6071486	total: 769ms	remaining: 2m 8s
11:	learn: 0.6022931	total: 839ms	remaining: 2m 8s
12:	learn: 0.5977593	total: 906ms	remaining: 2m 8s
13:	learn: 0.5936593	total: 980ms	remaining: 2m 8s
14:	learn: 0.5895303	total: 1.05s	remaining: 2m 8s
15:	learn: 0.5856031	total: 1.12s	remaining: 2m 8s
16:	learn: 0.5819175	total: 1.19s	remaining: 2m 8s
17:	learn: 0.5785914	total: 1.25s	remaining: 2m 7s
18:	learn: 0.5753810	total: 1.32s	remaining: 2m 7s
19:	learn: 0.5725716	total: 1.4s	

162:	learn: 0.5073935	total: 11.5s	remaining: 1m 59s
163:	learn: 0.5072920	total: 11.6s	remaining: 1m 59s
164:	learn: 0.5072069	total: 11.7s	remaining: 1m 58s
165:	learn: 0.5070932	total: 11.7s	remaining: 1m 58s
166:	learn: 0.5070308	total: 11.8s	remaining: 1m 58s
167:	learn: 0.5069322	total: 11.9s	remaining: 1m 58s
168:	learn: 0.5068675	total: 11.9s	remaining: 1m 58s
169:	learn: 0.5067544	total: 12s	remaining: 1m 58s
170:	learn: 0.5066294	total: 12.1s	remaining: 1m 58s
171:	learn: 0.5065498	total: 12.1s	remaining: 1m 58s
172:	learn: 0.5064609	total: 12.2s	remaining: 1m 58s
173:	learn: 0.5063631	total: 12.3s	remaining: 1m 58s
174:	learn: 0.5062796	total: 12.4s	remaining: 1m 58s
175:	learn: 0.5061865	total: 12.4s	remaining: 1m 58s
176:	learn: 0.5061183	total: 12.5s	remaining: 1m 58s
177:	learn: 0.5060412	total: 12.6s	remaining: 1m 58s
178:	learn: 0.5059719	total: 12.7s	remaining: 1m 58s
179:	learn: 0.5059075	total: 12.7s	remaining: 1m 58s
180:	learn: 0.5058603	total: 12.8s	remaining: 1m

319:	learn: 0.4972258	total: 22.7s	remaining: 1m 48s
320:	learn: 0.4971734	total: 22.8s	remaining: 1m 48s
321:	learn: 0.4971314	total: 22.9s	remaining: 1m 48s
322:	learn: 0.4970983	total: 23s	remaining: 1m 48s
323:	learn: 0.4970587	total: 23s	remaining: 1m 48s
324:	learn: 0.4970140	total: 23.1s	remaining: 1m 48s
325:	learn: 0.4969477	total: 23.2s	remaining: 1m 48s
326:	learn: 0.4969103	total: 23.3s	remaining: 1m 48s
327:	learn: 0.4968526	total: 23.3s	remaining: 1m 48s
328:	learn: 0.4968150	total: 23.4s	remaining: 1m 48s
329:	learn: 0.4967702	total: 23.5s	remaining: 1m 48s
330:	learn: 0.4967258	total: 23.5s	remaining: 1m 48s
331:	learn: 0.4966735	total: 23.6s	remaining: 1m 47s
332:	learn: 0.4966257	total: 23.7s	remaining: 1m 47s
333:	learn: 0.4965866	total: 23.8s	remaining: 1m 47s
334:	learn: 0.4965245	total: 23.8s	remaining: 1m 47s
335:	learn: 0.4964845	total: 23.9s	remaining: 1m 47s
336:	learn: 0.4964360	total: 24s	remaining: 1m 47s
337:	learn: 0.4963880	total: 24.1s	remaining: 1m 47s

475:	learn: 0.4897015	total: 34.1s	remaining: 1m 38s
476:	learn: 0.4896673	total: 34.2s	remaining: 1m 38s
477:	learn: 0.4896340	total: 34.3s	remaining: 1m 38s
478:	learn: 0.4895794	total: 34.3s	remaining: 1m 38s
479:	learn: 0.4895396	total: 34.4s	remaining: 1m 38s
480:	learn: 0.4894928	total: 34.5s	remaining: 1m 38s
481:	learn: 0.4894323	total: 34.6s	remaining: 1m 38s
482:	learn: 0.4893747	total: 34.6s	remaining: 1m 38s
483:	learn: 0.4893475	total: 34.7s	remaining: 1m 37s
484:	learn: 0.4893235	total: 34.8s	remaining: 1m 37s
485:	learn: 0.4892773	total: 34.9s	remaining: 1m 37s
486:	learn: 0.4892540	total: 34.9s	remaining: 1m 37s
487:	learn: 0.4892098	total: 35s	remaining: 1m 37s
488:	learn: 0.4891632	total: 35.1s	remaining: 1m 37s
489:	learn: 0.4891162	total: 35.2s	remaining: 1m 37s
490:	learn: 0.4890828	total: 35.2s	remaining: 1m 37s
491:	learn: 0.4890516	total: 35.3s	remaining: 1m 37s
492:	learn: 0.4890135	total: 35.4s	remaining: 1m 37s
493:	learn: 0.4889826	total: 35.4s	remaining: 1m

631:	learn: 0.4836293	total: 45.6s	remaining: 1m 27s
632:	learn: 0.4835942	total: 45.7s	remaining: 1m 27s
633:	learn: 0.4835511	total: 45.8s	remaining: 1m 27s
634:	learn: 0.4835267	total: 45.9s	remaining: 1m 27s
635:	learn: 0.4834882	total: 45.9s	remaining: 1m 27s
636:	learn: 0.4834399	total: 46s	remaining: 1m 27s
637:	learn: 0.4833934	total: 46.1s	remaining: 1m 27s
638:	learn: 0.4833576	total: 46.1s	remaining: 1m 27s
639:	learn: 0.4833113	total: 46.2s	remaining: 1m 27s
640:	learn: 0.4832668	total: 46.3s	remaining: 1m 27s
641:	learn: 0.4832342	total: 46.4s	remaining: 1m 27s
642:	learn: 0.4832187	total: 46.4s	remaining: 1m 27s
643:	learn: 0.4831677	total: 46.5s	remaining: 1m 27s
644:	learn: 0.4831267	total: 46.6s	remaining: 1m 27s
645:	learn: 0.4830902	total: 46.7s	remaining: 1m 26s
646:	learn: 0.4830481	total: 46.7s	remaining: 1m 26s
647:	learn: 0.4830215	total: 46.8s	remaining: 1m 26s
648:	learn: 0.4829848	total: 46.9s	remaining: 1m 26s
649:	learn: 0.4829522	total: 47s	remaining: 1m 2

...

1102:	learn: 0.4704686	total: 1m 21s	remaining: 55.1s
1103:	learn: 0.4704393	total: 1m 21s	remaining: 55s
1104:	learn: 0.4704142	total: 1m 21s	remaining: 55s
1105:	learn: 0.4703755	total: 1m 21s	remaining: 54.9s
1106:	learn: 0.4703754	total: 1m 21s	remaining: 54.8s
1107:	learn: 0.4703745	total: 1m 21s	remaining: 54.8s
1108:	learn: 0.4703524	total: 1m 21s	remaining: 54.7s
1109:	learn: 0.4703382	total: 1m 21s	remaining: 54.6s
1110:	learn: 0.4703101	total: 1m 22s	remaining: 54.6s
1111:	learn: 0.4703045	total: 1m 22s	remaining: 54.5s
1112:	learn: 0.4702888	total: 1m 22s	remaining: 54.4s
1113:	learn: 0.4702823	total: 1m 22s	remaining: 54.4s
1114:	learn: 0.4702475	total: 1m 22s	remaining: 54.3s
1115:	learn: 0.4702199	total: 1m 22s	remaining: 54.2s
1116:	learn: 0.4701999	total: 1m 22s	remaining: 54.2s
1117:	learn: 0.4701752	total: 1m 22s	remaining: 54.1s
1118:	learn: 0.4701576	total: 1m 22s	remaining: 54s
1119:	learn: 0.4701461	total: 1m 22s	remaining: 53.9s
1120:	learn: 0.4701300	total: 1m 2

1255:	learn: 0.4672933	total: 1m 32s	remaining: 43.9s
1256:	learn: 0.4672724	total: 1m 32s	remaining: 43.8s
1257:	learn: 0.4672535	total: 1m 33s	remaining: 43.8s
1258:	learn: 0.4672367	total: 1m 33s	remaining: 43.7s
1259:	learn: 0.4672170	total: 1m 33s	remaining: 43.6s
1260:	learn: 0.4672025	total: 1m 33s	remaining: 43.6s
1261:	learn: 0.4671771	total: 1m 33s	remaining: 43.5s
1262:	learn: 0.4671705	total: 1m 33s	remaining: 43.4s
1263:	learn: 0.4671516	total: 1m 33s	remaining: 43.3s
1264:	learn: 0.4671199	total: 1m 33s	remaining: 43.3s
1265:	learn: 0.4670760	total: 1m 33s	remaining: 43.2s
1266:	learn: 0.4670478	total: 1m 33s	remaining: 43.1s
1267:	learn: 0.4670337	total: 1m 33s	remaining: 43s
1268:	learn: 0.4670249	total: 1m 33s	remaining: 43s
1269:	learn: 0.4669897	total: 1m 33s	remaining: 42.9s
1270:	learn: 0.4669663	total: 1m 33s	remaining: 42.8s
1271:	learn: 0.4669433	total: 1m 34s	remaining: 42.7s
1272:	learn: 0.4669190	total: 1m 34s	remaining: 42.7s
1273:	learn: 0.4668822	total: 1m

1408:	learn: 0.4639497	total: 1m 44s	remaining: 32.6s
1409:	learn: 0.4639218	total: 1m 44s	remaining: 32.6s
1410:	learn: 0.4638901	total: 1m 44s	remaining: 32.5s
1411:	learn: 0.4638680	total: 1m 44s	remaining: 32.4s
1412:	learn: 0.4638359	total: 1m 44s	remaining: 32.4s
1413:	learn: 0.4638101	total: 1m 44s	remaining: 32.3s
1414:	learn: 0.4638077	total: 1m 44s	remaining: 32.2s
1415:	learn: 0.4637925	total: 1m 44s	remaining: 32.1s
1416:	learn: 0.4637588	total: 1m 44s	remaining: 32.1s
1417:	learn: 0.4637299	total: 1m 44s	remaining: 32s
1418:	learn: 0.4637037	total: 1m 45s	remaining: 31.9s
1419:	learn: 0.4636931	total: 1m 45s	remaining: 31.8s
1420:	learn: 0.4636730	total: 1m 45s	remaining: 31.8s
1421:	learn: 0.4636519	total: 1m 45s	remaining: 31.7s
1422:	learn: 0.4636336	total: 1m 45s	remaining: 31.6s
1423:	learn: 0.4636327	total: 1m 45s	remaining: 31.5s
1424:	learn: 0.4635986	total: 1m 45s	remaining: 31.5s
1425:	learn: 0.4635812	total: 1m 45s	remaining: 31.4s
1426:	learn: 0.4635575	total: 

1561:	learn: 0.4605323	total: 1m 55s	remaining: 21.3s
1562:	learn: 0.4604937	total: 1m 55s	remaining: 21.3s
1563:	learn: 0.4604523	total: 1m 55s	remaining: 21.2s
1564:	learn: 0.4604315	total: 1m 55s	remaining: 21.1s
1565:	learn: 0.4604082	total: 1m 56s	remaining: 21s
1566:	learn: 0.4603783	total: 1m 56s	remaining: 21s
1567:	learn: 0.4603570	total: 1m 56s	remaining: 20.9s
1568:	learn: 0.4603378	total: 1m 56s	remaining: 20.8s
1569:	learn: 0.4603130	total: 1m 56s	remaining: 20.7s
1570:	learn: 0.4602875	total: 1m 56s	remaining: 20.7s
1571:	learn: 0.4602700	total: 1m 56s	remaining: 20.6s
1572:	learn: 0.4602469	total: 1m 56s	remaining: 20.5s
1573:	learn: 0.4602252	total: 1m 56s	remaining: 20.5s
1574:	learn: 0.4602238	total: 1m 56s	remaining: 20.4s
1575:	learn: 0.4602204	total: 1m 56s	remaining: 20.3s
1576:	learn: 0.4602015	total: 1m 56s	remaining: 20.2s
1577:	learn: 0.4601755	total: 1m 56s	remaining: 20.2s
1578:	learn: 0.4601391	total: 1m 56s	remaining: 20.1s
1579:	learn: 0.4601110	total: 1m

1718:	learn: 0.4572943	total: 2m 7s	remaining: 9.72s
1719:	learn: 0.4572708	total: 2m 7s	remaining: 9.64s
1720:	learn: 0.4572548	total: 2m 7s	remaining: 9.57s
1721:	learn: 0.4572376	total: 2m 7s	remaining: 9.5s
1722:	learn: 0.4572242	total: 2m 7s	remaining: 9.43s
1723:	learn: 0.4572099	total: 2m 7s	remaining: 9.35s
1724:	learn: 0.4571894	total: 2m 8s	remaining: 9.28s
1725:	learn: 0.4571538	total: 2m 8s	remaining: 9.2s
1726:	learn: 0.4571347	total: 2m 8s	remaining: 9.13s
1727:	learn: 0.4571165	total: 2m 8s	remaining: 9.06s
1728:	learn: 0.4570726	total: 2m 8s	remaining: 8.98s
1729:	learn: 0.4570599	total: 2m 8s	remaining: 8.91s
1730:	learn: 0.4570496	total: 2m 8s	remaining: 8.84s
1731:	learn: 0.4570388	total: 2m 8s	remaining: 8.76s
1732:	learn: 0.4570215	total: 2m 8s	remaining: 8.69s
1733:	learn: 0.4570142	total: 2m 8s	remaining: 8.61s
1734:	learn: 0.4569973	total: 2m 8s	remaining: 8.54s
1735:	learn: 0.4569744	total: 2m 8s	remaining: 8.46s
1736:	learn: 0.4569548	total: 2m 8s	remaining: 8

## <span style="color:purple"> Prédiction sur le jeu de test </span>

In [31]:
probabilities = catboost_model.predict_proba(test_officiel)[:, 1]
df_probabilites = pd.DataFrame(probabilities, columns=['GRAVE'])
df_result_catboost = pd.concat([Num_Acc_test, df_probabilites], axis=1)
df_result_catboost

,Num_Acc,GRAVE
0,201200049538,0.126035
1,201200004221,0.919449
2,201200002457,0.931213
3,201200009834,0.827554
4,201200025602,0.260982
...,...,...
63539,202200008772,0.261321
63540,202200026268,0.079851
63541,202200043480,0.574100
63542,202200022087,0.269083


In [32]:
# Écrire le DataFrame dans un fichier CSV
df_result_catboost.to_csv('catboost_classifier_12_50.csv', index=False)

### <span style="color:orange">Votting classifier (modèle 2)

In [33]:
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


# lbgm classifier 
lgbm_params = {'learning_rate': 0.075, 'n_estimators': 255, 'num_leaves': 225}
lgb_model = lgb.LGBMClassifier(**lgbm_params)

# xgb model
#(max_depth=10,learning_rate=0.06, n_estimators=450, gamma=0.03, objective='binary:logistic')
xgb_params = {'max_depth':10, 'learning_rate':0.06, 'n_estimators':450, 'gamma':0.03, 'objective':'binary:logistic'}
xgb_model = xgb.XGBClassifier(**xgb_params)

# Définir les paramètres pour Gradient Boosting
gb_params = {'n_estimators': 191, 'learning_rate': 0.07, 'max_depth': 7, 'random_state': 42}
gradient_boosting_model = GradientBoostingClassifier(**gb_params)

# catboost
catb_params = {'depth': 10, 'iterations': 1850, 'l2_leaf_reg': 7, 'learning_rate': 0.0325}
catboost_model =CatBoostClassifier(**catb_params)


# Créer le Voting Classifier avec les modèles spécifiés et leurs paramètres
voting_clf = VotingClassifier(estimators=[
    ('catb', catboost_model),
    ('lgbm', lgb_model),
    ('xgb',xgb_model),
    ('gradient_boosting', gradient_boosting_model)
], voting='soft')  # Utiliser soft voting pour prédire les probabilités

# Entraîner le Voting Classifier
voting_clf.fit(X_train, y_train)

# Évaluer les performances du Voting Classifier
accuracy = voting_clf.score(X_test, y_test)
print("Accuracy:", accuracy)

probabilities = voting_clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, probabilities)
print("AUC Score:", auc)


0:	learn: 0.6826228	total: 76.8ms	remaining: 2m 21s
1:	learn: 0.6722128	total: 146ms	remaining: 2m 15s
2:	learn: 0.6625923	total: 214ms	remaining: 2m 11s
3:	learn: 0.6536300	total: 283ms	remaining: 2m 10s
4:	learn: 0.6455832	total: 355ms	remaining: 2m 11s
5:	learn: 0.6379329	total: 423ms	remaining: 2m 10s
6:	learn: 0.6307800	total: 491ms	remaining: 2m 9s
7:	learn: 0.6242668	total: 563ms	remaining: 2m 9s
8:	learn: 0.6183424	total: 630ms	remaining: 2m 8s
9:	learn: 0.6125395	total: 698ms	remaining: 2m 8s
10:	learn: 0.6071486	total: 766ms	remaining: 2m 8s
11:	learn: 0.6022931	total: 835ms	remaining: 2m 7s
12:	learn: 0.5977593	total: 904ms	remaining: 2m 7s
13:	learn: 0.5936593	total: 977ms	remaining: 2m 8s
14:	learn: 0.5895303	total: 1.04s	remaining: 2m 7s
15:	learn: 0.5856031	total: 1.11s	remaining: 2m 7s
16:	learn: 0.5819175	total: 1.18s	remaining: 2m 7s
17:	learn: 0.5785914	total: 1.25s	remaining: 2m 7s
18:	learn: 0.5753810	total: 1.32s	remaining: 2m 7s
19:	learn: 0.5725716	total: 1.4s	r

160:	learn: 0.5075778	total: 12s	remaining: 2m 5s
161:	learn: 0.5074812	total: 12s	remaining: 2m 5s
162:	learn: 0.5073935	total: 12.1s	remaining: 2m 5s
163:	learn: 0.5072920	total: 12.2s	remaining: 2m 5s
164:	learn: 0.5072069	total: 12.3s	remaining: 2m 5s
165:	learn: 0.5070932	total: 12.3s	remaining: 2m 5s
166:	learn: 0.5070308	total: 12.4s	remaining: 2m 5s
167:	learn: 0.5069322	total: 12.5s	remaining: 2m 5s
168:	learn: 0.5068675	total: 12.6s	remaining: 2m 5s
169:	learn: 0.5067544	total: 12.6s	remaining: 2m 4s
170:	learn: 0.5066294	total: 12.7s	remaining: 2m 4s
171:	learn: 0.5065498	total: 12.8s	remaining: 2m 4s
172:	learn: 0.5064609	total: 12.9s	remaining: 2m 4s
173:	learn: 0.5063631	total: 12.9s	remaining: 2m 4s
174:	learn: 0.5062796	total: 13s	remaining: 2m 4s
175:	learn: 0.5061865	total: 13.1s	remaining: 2m 4s
176:	learn: 0.5061183	total: 13.1s	remaining: 2m 4s
177:	learn: 0.5060412	total: 13.2s	remaining: 2m 4s
178:	learn: 0.5059719	total: 13.3s	remaining: 2m 3s
179:	learn: 0.5059

317:	learn: 0.4973537	total: 23.4s	remaining: 1m 52s
318:	learn: 0.4972896	total: 23.5s	remaining: 1m 52s
319:	learn: 0.4972258	total: 23.6s	remaining: 1m 52s
320:	learn: 0.4971734	total: 23.6s	remaining: 1m 52s
321:	learn: 0.4971314	total: 23.7s	remaining: 1m 52s
322:	learn: 0.4970983	total: 23.8s	remaining: 1m 52s
323:	learn: 0.4970587	total: 23.9s	remaining: 1m 52s
324:	learn: 0.4970140	total: 24s	remaining: 1m 52s
325:	learn: 0.4969477	total: 24s	remaining: 1m 52s
326:	learn: 0.4969103	total: 24.1s	remaining: 1m 52s
327:	learn: 0.4968526	total: 24.2s	remaining: 1m 52s
328:	learn: 0.4968150	total: 24.3s	remaining: 1m 52s
329:	learn: 0.4967702	total: 24.4s	remaining: 1m 52s
330:	learn: 0.4967258	total: 24.4s	remaining: 1m 52s
331:	learn: 0.4966735	total: 24.5s	remaining: 1m 52s
332:	learn: 0.4966257	total: 24.6s	remaining: 1m 52s
333:	learn: 0.4965866	total: 24.7s	remaining: 1m 51s
334:	learn: 0.4965245	total: 24.8s	remaining: 1m 51s
335:	learn: 0.4964845	total: 24.8s	remaining: 1m 5

473:	learn: 0.4898126	total: 35s	remaining: 1m 41s
474:	learn: 0.4897612	total: 35.1s	remaining: 1m 41s
475:	learn: 0.4897015	total: 35.1s	remaining: 1m 41s
476:	learn: 0.4896673	total: 35.2s	remaining: 1m 41s
477:	learn: 0.4896340	total: 35.3s	remaining: 1m 41s
478:	learn: 0.4895794	total: 35.3s	remaining: 1m 41s
479:	learn: 0.4895396	total: 35.4s	remaining: 1m 41s
480:	learn: 0.4894928	total: 35.5s	remaining: 1m 41s
481:	learn: 0.4894323	total: 35.6s	remaining: 1m 40s
482:	learn: 0.4893747	total: 35.6s	remaining: 1m 40s
483:	learn: 0.4893475	total: 35.7s	remaining: 1m 40s
484:	learn: 0.4893235	total: 35.8s	remaining: 1m 40s
485:	learn: 0.4892773	total: 35.8s	remaining: 1m 40s
486:	learn: 0.4892540	total: 35.9s	remaining: 1m 40s
487:	learn: 0.4892098	total: 36s	remaining: 1m 40s
488:	learn: 0.4891632	total: 36.1s	remaining: 1m 40s
489:	learn: 0.4891162	total: 36.1s	remaining: 1m 40s
490:	learn: 0.4890828	total: 36.2s	remaining: 1m 40s
491:	learn: 0.4890516	total: 36.3s	remaining: 1m 4

630:	learn: 0.4836543	total: 46.3s	remaining: 1m 29s
631:	learn: 0.4836293	total: 46.4s	remaining: 1m 29s
632:	learn: 0.4835942	total: 46.5s	remaining: 1m 29s
633:	learn: 0.4835511	total: 46.5s	remaining: 1m 29s
634:	learn: 0.4835267	total: 46.6s	remaining: 1m 29s
635:	learn: 0.4834882	total: 46.7s	remaining: 1m 29s
636:	learn: 0.4834399	total: 46.8s	remaining: 1m 29s
637:	learn: 0.4833934	total: 46.8s	remaining: 1m 28s
638:	learn: 0.4833576	total: 46.9s	remaining: 1m 28s
639:	learn: 0.4833113	total: 47s	remaining: 1m 28s
640:	learn: 0.4832668	total: 47s	remaining: 1m 28s
641:	learn: 0.4832342	total: 47.1s	remaining: 1m 28s
642:	learn: 0.4832187	total: 47.2s	remaining: 1m 28s
643:	learn: 0.4831677	total: 47.3s	remaining: 1m 28s
644:	learn: 0.4831267	total: 47.3s	remaining: 1m 28s
645:	learn: 0.4830902	total: 47.4s	remaining: 1m 28s
646:	learn: 0.4830481	total: 47.5s	remaining: 1m 28s
647:	learn: 0.4830215	total: 47.5s	remaining: 1m 28s
648:	learn: 0.4829848	total: 47.6s	remaining: 1m 2

...

1408:	learn: 0.4639497	total: 1m 42s	remaining: 32s
1409:	learn: 0.4639218	total: 1m 42s	remaining: 31.9s
1410:	learn: 0.4638901	total: 1m 42s	remaining: 31.8s
1411:	learn: 0.4638680	total: 1m 42s	remaining: 31.8s
1412:	learn: 0.4638359	total: 1m 42s	remaining: 31.7s
1413:	learn: 0.4638101	total: 1m 42s	remaining: 31.6s
1414:	learn: 0.4638077	total: 1m 42s	remaining: 31.5s
1415:	learn: 0.4637925	total: 1m 42s	remaining: 31.5s
1416:	learn: 0.4637588	total: 1m 42s	remaining: 31.4s
1417:	learn: 0.4637299	total: 1m 42s	remaining: 31.3s
1418:	learn: 0.4637037	total: 1m 42s	remaining: 31.3s
1419:	learn: 0.4636931	total: 1m 42s	remaining: 31.2s
1420:	learn: 0.4636730	total: 1m 43s	remaining: 31.1s
1421:	learn: 0.4636519	total: 1m 43s	remaining: 31s
1422:	learn: 0.4636336	total: 1m 43s	remaining: 31s
1423:	learn: 0.4636327	total: 1m 43s	remaining: 30.9s
1424:	learn: 0.4635986	total: 1m 43s	remaining: 30.8s
1425:	learn: 0.4635812	total: 1m 43s	remaining: 30.7s
1426:	learn: 0.4635575	total: 1m 4

1561:	learn: 0.4605323	total: 1m 53s	remaining: 20.9s
1562:	learn: 0.4604937	total: 1m 53s	remaining: 20.8s
1563:	learn: 0.4604523	total: 1m 53s	remaining: 20.7s
1564:	learn: 0.4604315	total: 1m 53s	remaining: 20.6s
1565:	learn: 0.4604082	total: 1m 53s	remaining: 20.6s
1566:	learn: 0.4603783	total: 1m 53s	remaining: 20.5s
1567:	learn: 0.4603570	total: 1m 53s	remaining: 20.4s
1568:	learn: 0.4603378	total: 1m 53s	remaining: 20.4s
1569:	learn: 0.4603130	total: 1m 53s	remaining: 20.3s
1570:	learn: 0.4602875	total: 1m 53s	remaining: 20.2s
1571:	learn: 0.4602700	total: 1m 53s	remaining: 20.1s
1572:	learn: 0.4602469	total: 1m 53s	remaining: 20.1s
1573:	learn: 0.4602252	total: 1m 54s	remaining: 20s
1574:	learn: 0.4602238	total: 1m 54s	remaining: 19.9s
1575:	learn: 0.4602204	total: 1m 54s	remaining: 19.8s
1576:	learn: 0.4602015	total: 1m 54s	remaining: 19.8s
1577:	learn: 0.4601755	total: 1m 54s	remaining: 19.7s
1578:	learn: 0.4601391	total: 1m 54s	remaining: 19.6s
1579:	learn: 0.4601110	total: 

1716:	learn: 0.4573585	total: 2m 4s	remaining: 9.63s
1717:	learn: 0.4573254	total: 2m 4s	remaining: 9.55s
1718:	learn: 0.4572943	total: 2m 4s	remaining: 9.48s
1719:	learn: 0.4572708	total: 2m 4s	remaining: 9.41s
1720:	learn: 0.4572548	total: 2m 4s	remaining: 9.34s
1721:	learn: 0.4572376	total: 2m 4s	remaining: 9.26s
1722:	learn: 0.4572242	total: 2m 4s	remaining: 9.19s
1723:	learn: 0.4572099	total: 2m 4s	remaining: 9.12s
1724:	learn: 0.4571894	total: 2m 4s	remaining: 9.05s
1725:	learn: 0.4571538	total: 2m 4s	remaining: 8.97s
1726:	learn: 0.4571347	total: 2m 4s	remaining: 8.9s
1727:	learn: 0.4571165	total: 2m 5s	remaining: 8.83s
1728:	learn: 0.4570726	total: 2m 5s	remaining: 8.76s
1729:	learn: 0.4570599	total: 2m 5s	remaining: 8.68s
1730:	learn: 0.4570496	total: 2m 5s	remaining: 8.61s
1731:	learn: 0.4570388	total: 2m 5s	remaining: 8.54s
1732:	learn: 0.4570215	total: 2m 5s	remaining: 8.47s
1733:	learn: 0.4570142	total: 2m 5s	remaining: 8.39s
1734:	learn: 0.4569973	total: 2m 5s	remaining: 

C:\Apps\Anaconda\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Apps\Anaconda\lib\site-packages\joblib\externals\loky\backend\context.py", line 245, in _count_physical_cores
    raise ValueError(


[LightGBM] [Info] Number of positive: 204134, number of negative: 291664
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 868
[LightGBM] [Info] Number of data points in the train set: 495798, number of used features: 67
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.411728 -> initscore=-0.356826
[LightGBM] [Info] Start training from score -0.356826
Accuracy: 0.7617254783844583
AUC Score: 0.8296822766693219


## <span style="color:purple">Prédiction sur le jeu de test</span>

In [34]:
probabilities = voting_clf.predict_proba(test_officiel)[:, 1]
df_probabilites = pd.DataFrame(probabilities, columns=['GRAVE'])
df_result_voting = pd.concat([Num_Acc_test, df_probabilites], axis=1)
df_result_voting

,Num_Acc,GRAVE
0,201200049538,0.171993
1,201200004221,0.933110
2,201200002457,0.943268
3,201200009834,0.828704
4,201200025602,0.255046
...,...,...
63539,202200008772,0.284002
63540,202200026268,0.081740
63541,202200043480,0.619463
63542,202200022087,0.370604


In [35]:
# Écrire le DataFrame dans un fichier CSV
df_result_voting.to_csv('votting_classifier_12_51.csv', index=False)

### <span style="color:orange"> On fait un blender classifier (modèle 3)</span>

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
import numpy as np
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# Diviser les données en ensembles d'entraînement, de validation et de test
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Définir les paramètres pour le LGBMClassifier
lgbm_params = {'learning_rate': 0.075, 'n_estimators': 255, 'num_leaves': 225}
lgb_model = lgb.LGBMClassifier(**lgbm_params)

# Définir les paramètres pour le XGBClassifier
xgb_params = {'max_depth':10, 'learning_rate':0.06, 'n_estimators':450, 'gamma':0.03, 'objective':'binary:logistic'}
xgb_model = xgb.XGBClassifier(**xgb_params)

# catboost
catb_params = {'depth': 10, 'iterations': 1850, 'l2_leaf_reg': 7, 'learning_rate': 0.0325}
catboost_model =CatBoostClassifier(**catb_params)

#gradient boosting
# Définir les paramètres pour Gradient Boosting
gb_params = {'n_estimators': 191, 'learning_rate': 0.07, 'max_depth': 7, 'random_state': 42}

# Initialiser le modèle de Gradient Boosting avec les paramètres spécifiés
gradient_boosting_model = GradientBoostingClassifier(**gb_params)

# Entraîner les modèles de base
catboost_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
gradient_boosting_model.fit(X_train, y_train)


# Faire des prédictions sur l'ensemble de validation avec les modèles de base
lgbm_pred_val = lgb_model.predict_proba(X_val)[:, 1]
xgb_pred_val = xgb_model.predict_proba(X_val)[:, 1]
catb_pred_val = catboost_model.predict_proba(X_val)[:, 1]
gradient_boosting_pred_val = gradient_boosting_model.predict_proba(X_val)[:, 1]

# Utiliser les prédictions des modèles de base comme caractéristiques d'entrée pour un modèle de méta-apprentissage
X_blend_val = np.column_stack((lgbm_pred_val, xgb_pred_val, catb_pred_val, gradient_boosting_pred_val))

# Entraîner un modèle de méta-apprentissage (par exemple, une régression logistique) sur les prédictions des modèles de base
params_logReg = {'C': 1, 'penalty': 'l2'}
blend_model = LogisticRegression(**params_logReg)
blend_model.fit(X_blend_val, y_val)

# Faire des prédictions sur de nouvelles données en utilisant le modèle de blending
lgbm_pred_test = lgb_model.predict_proba(X_test)[:, 1]
xgb_pred_test = xgb_model.predict_proba(X_test)[:, 1]
catb_pred_test = catboost_model.predict_proba(X_test)[:, 1]
gradient_boosting_pred_test = gradient_boosting_model.predict_proba(X_test)[:, 1]

X_blend_test = np.column_stack((lgbm_pred_test, xgb_pred_test, catb_pred_test, gradient_boosting_pred_test))
y_pred_blend = blend_model.predict_proba(X_blend_test)[:, 1]

# Évaluer les performances du modèle de blending
auc_blend = roc_auc_score(y_test, y_pred_blend)
print("AUC Score avec blending sur l'ensemble de test:", auc_blend)

0:	learn: 0.6826155	total: 74.5ms	remaining: 2m 17s
1:	learn: 0.6724780	total: 140ms	remaining: 2m 9s
2:	learn: 0.6627922	total: 210ms	remaining: 2m 8s
3:	learn: 0.6538339	total: 277ms	remaining: 2m 7s
4:	learn: 0.6455490	total: 346ms	remaining: 2m 7s
5:	learn: 0.6379230	total: 414ms	remaining: 2m 7s
6:	learn: 0.6306982	total: 480ms	remaining: 2m 6s
7:	learn: 0.6241703	total: 544ms	remaining: 2m 5s
8:	learn: 0.6182229	total: 612ms	remaining: 2m 5s
9:	learn: 0.6125062	total: 683ms	remaining: 2m 5s
10:	learn: 0.6070854	total: 755ms	remaining: 2m 6s
11:	learn: 0.6022029	total: 830ms	remaining: 2m 7s
12:	learn: 0.5974487	total: 898ms	remaining: 2m 6s
13:	learn: 0.5931324	total: 966ms	remaining: 2m 6s
14:	learn: 0.5892185	total: 1.03s	remaining: 2m 6s
15:	learn: 0.5855519	total: 1.11s	remaining: 2m 7s
16:	learn: 0.5818428	total: 1.18s	remaining: 2m 7s
17:	learn: 0.5783954	total: 1.25s	remaining: 2m 6s
18:	learn: 0.5752015	total: 1.31s	remaining: 2m 6s
19:	learn: 0.5724320	total: 1.38s	remai

163:	learn: 0.5068983	total: 11.3s	remaining: 1m 56s
164:	learn: 0.5068067	total: 11.4s	remaining: 1m 56s
165:	learn: 0.5066755	total: 11.5s	remaining: 1m 56s
166:	learn: 0.5065998	total: 11.5s	remaining: 1m 56s
167:	learn: 0.5065062	total: 11.6s	remaining: 1m 56s
168:	learn: 0.5064294	total: 11.7s	remaining: 1m 56s
169:	learn: 0.5062932	total: 11.7s	remaining: 1m 56s
170:	learn: 0.5061430	total: 11.8s	remaining: 1m 55s
171:	learn: 0.5060664	total: 11.9s	remaining: 1m 55s
172:	learn: 0.5059644	total: 12s	remaining: 1m 55s
173:	learn: 0.5058881	total: 12s	remaining: 1m 55s
174:	learn: 0.5058165	total: 12.1s	remaining: 1m 55s
175:	learn: 0.5057519	total: 12.2s	remaining: 1m 55s
176:	learn: 0.5056563	total: 12.2s	remaining: 1m 55s
177:	learn: 0.5055369	total: 12.3s	remaining: 1m 55s
178:	learn: 0.5054442	total: 12.4s	remaining: 1m 55s
179:	learn: 0.5053744	total: 12.4s	remaining: 1m 55s
180:	learn: 0.5053186	total: 12.5s	remaining: 1m 55s
181:	learn: 0.5052298	total: 12.6s	remaining: 1m 5

319:	learn: 0.4964477	total: 22.1s	remaining: 1m 45s
320:	learn: 0.4963898	total: 22.1s	remaining: 1m 45s
321:	learn: 0.4963445	total: 22.2s	remaining: 1m 45s
322:	learn: 0.4963042	total: 22.3s	remaining: 1m 45s
323:	learn: 0.4962835	total: 22.3s	remaining: 1m 45s
324:	learn: 0.4962279	total: 22.4s	remaining: 1m 45s
325:	learn: 0.4961665	total: 22.5s	remaining: 1m 45s
326:	learn: 0.4961001	total: 22.6s	remaining: 1m 45s
327:	learn: 0.4960433	total: 22.6s	remaining: 1m 44s
328:	learn: 0.4960047	total: 22.7s	remaining: 1m 44s
329:	learn: 0.4959645	total: 22.8s	remaining: 1m 44s
330:	learn: 0.4958991	total: 22.8s	remaining: 1m 44s
331:	learn: 0.4958377	total: 22.9s	remaining: 1m 44s
332:	learn: 0.4957975	total: 23s	remaining: 1m 44s
333:	learn: 0.4957721	total: 23s	remaining: 1m 44s
334:	learn: 0.4956929	total: 23.1s	remaining: 1m 44s
335:	learn: 0.4956545	total: 23.2s	remaining: 1m 44s
336:	learn: 0.4956179	total: 23.2s	remaining: 1m 44s
337:	learn: 0.4955720	total: 23.3s	remaining: 1m 4

477:	learn: 0.4879805	total: 33.2s	remaining: 1m 35s
478:	learn: 0.4879242	total: 33.2s	remaining: 1m 35s
479:	learn: 0.4878850	total: 33.3s	remaining: 1m 35s
480:	learn: 0.4878221	total: 33.4s	remaining: 1m 34s
481:	learn: 0.4877558	total: 33.4s	remaining: 1m 34s
482:	learn: 0.4877024	total: 33.5s	remaining: 1m 34s
483:	learn: 0.4876477	total: 33.6s	remaining: 1m 34s
484:	learn: 0.4875920	total: 33.7s	remaining: 1m 34s
485:	learn: 0.4875215	total: 33.7s	remaining: 1m 34s
486:	learn: 0.4874713	total: 33.8s	remaining: 1m 34s
487:	learn: 0.4874168	total: 33.9s	remaining: 1m 34s
488:	learn: 0.4873652	total: 33.9s	remaining: 1m 34s
489:	learn: 0.4873128	total: 34s	remaining: 1m 34s
490:	learn: 0.4872721	total: 34.1s	remaining: 1m 34s
491:	learn: 0.4872284	total: 34.1s	remaining: 1m 34s
492:	learn: 0.4871857	total: 34.2s	remaining: 1m 34s
493:	learn: 0.4871453	total: 34.3s	remaining: 1m 34s
494:	learn: 0.4870918	total: 34.4s	remaining: 1m 34s
495:	learn: 0.4870230	total: 34.4s	remaining: 1m

634:	learn: 0.4810598	total: 44.3s	remaining: 1m 24s
635:	learn: 0.4810192	total: 44.3s	remaining: 1m 24s
636:	learn: 0.4809730	total: 44.4s	remaining: 1m 24s
637:	learn: 0.4809279	total: 44.5s	remaining: 1m 24s
638:	learn: 0.4808887	total: 44.5s	remaining: 1m 24s
639:	learn: 0.4808253	total: 44.6s	remaining: 1m 24s
640:	learn: 0.4807855	total: 44.7s	remaining: 1m 24s
641:	learn: 0.4807386	total: 44.8s	remaining: 1m 24s
642:	learn: 0.4806976	total: 44.8s	remaining: 1m 24s
643:	learn: 0.4806618	total: 44.9s	remaining: 1m 24s
644:	learn: 0.4806320	total: 45s	remaining: 1m 24s
645:	learn: 0.4805949	total: 45s	remaining: 1m 23s
646:	learn: 0.4805642	total: 45.1s	remaining: 1m 23s
647:	learn: 0.4805245	total: 45.2s	remaining: 1m 23s
648:	learn: 0.4804977	total: 45.3s	remaining: 1m 23s
649:	learn: 0.4804536	total: 45.3s	remaining: 1m 23s
650:	learn: 0.4804027	total: 45.4s	remaining: 1m 23s
651:	learn: 0.4803636	total: 45.5s	remaining: 1m 23s
652:	learn: 0.4803445	total: 45.5s	remaining: 1m 2

...

1107:	learn: 0.4672145	total: 1m 17s	remaining: 52s
1108:	learn: 0.4671703	total: 1m 17s	remaining: 51.9s
1109:	learn: 0.4671702	total: 1m 17s	remaining: 51.8s
1110:	learn: 0.4671565	total: 1m 17s	remaining: 51.8s
1111:	learn: 0.4671227	total: 1m 17s	remaining: 51.7s
1112:	learn: 0.4670923	total: 1m 17s	remaining: 51.6s
1113:	learn: 0.4670706	total: 1m 18s	remaining: 51.6s
1114:	learn: 0.4670484	total: 1m 18s	remaining: 51.5s
1115:	learn: 0.4670238	total: 1m 18s	remaining: 51.4s
1116:	learn: 0.4669875	total: 1m 18s	remaining: 51.4s
1117:	learn: 0.4669723	total: 1m 18s	remaining: 51.3s
1118:	learn: 0.4669608	total: 1m 18s	remaining: 51.2s
1119:	learn: 0.4669277	total: 1m 18s	remaining: 51.2s
1120:	learn: 0.4669076	total: 1m 18s	remaining: 51.1s
1121:	learn: 0.4668763	total: 1m 18s	remaining: 51s
1122:	learn: 0.4668728	total: 1m 18s	remaining: 51s
1123:	learn: 0.4668516	total: 1m 18s	remaining: 50.9s
1124:	learn: 0.4668233	total: 1m 18s	remaining: 50.8s
1125:	learn: 0.4668152	total: 1m 1

1260:	learn: 0.4634188	total: 1m 28s	remaining: 41.4s
1261:	learn: 0.4633967	total: 1m 28s	remaining: 41.3s
1262:	learn: 0.4633628	total: 1m 28s	remaining: 41.3s
1263:	learn: 0.4633206	total: 1m 28s	remaining: 41.2s
1264:	learn: 0.4633087	total: 1m 28s	remaining: 41.1s
1265:	learn: 0.4632814	total: 1m 28s	remaining: 41s
1266:	learn: 0.4632573	total: 1m 29s	remaining: 41s
1267:	learn: 0.4632169	total: 1m 29s	remaining: 40.9s
1268:	learn: 0.4631997	total: 1m 29s	remaining: 40.8s
1269:	learn: 0.4631723	total: 1m 29s	remaining: 40.8s
1270:	learn: 0.4631482	total: 1m 29s	remaining: 40.7s
1271:	learn: 0.4631223	total: 1m 29s	remaining: 40.6s
1272:	learn: 0.4631092	total: 1m 29s	remaining: 40.6s
1273:	learn: 0.4630809	total: 1m 29s	remaining: 40.5s
1274:	learn: 0.4630533	total: 1m 29s	remaining: 40.4s
1275:	learn: 0.4630295	total: 1m 29s	remaining: 40.3s
1276:	learn: 0.4629936	total: 1m 29s	remaining: 40.3s
1277:	learn: 0.4629694	total: 1m 29s	remaining: 40.2s
1278:	learn: 0.4629611	total: 1m

1414:	learn: 0.4598822	total: 1m 39s	remaining: 30.6s
1415:	learn: 0.4598404	total: 1m 39s	remaining: 30.6s
1416:	learn: 0.4597995	total: 1m 39s	remaining: 30.5s
1417:	learn: 0.4597592	total: 1m 39s	remaining: 30.4s
1418:	learn: 0.4597367	total: 1m 39s	remaining: 30.4s
1419:	learn: 0.4597176	total: 1m 40s	remaining: 30.3s
1420:	learn: 0.4596833	total: 1m 40s	remaining: 30.2s
1421:	learn: 0.4596630	total: 1m 40s	remaining: 30.1s
1422:	learn: 0.4596373	total: 1m 40s	remaining: 30.1s
1423:	learn: 0.4595977	total: 1m 40s	remaining: 30s
1424:	learn: 0.4595669	total: 1m 40s	remaining: 29.9s
1425:	learn: 0.4595433	total: 1m 40s	remaining: 29.9s
1426:	learn: 0.4595066	total: 1m 40s	remaining: 29.8s
1427:	learn: 0.4594589	total: 1m 40s	remaining: 29.7s
1428:	learn: 0.4594457	total: 1m 40s	remaining: 29.7s
1429:	learn: 0.4594177	total: 1m 40s	remaining: 29.6s
1430:	learn: 0.4593876	total: 1m 40s	remaining: 29.5s
1431:	learn: 0.4593665	total: 1m 40s	remaining: 29.4s
1432:	learn: 0.4593551	total: 

1569:	learn: 0.4563243	total: 1m 50s	remaining: 19.8s
1570:	learn: 0.4563017	total: 1m 50s	remaining: 19.7s
1571:	learn: 0.4562729	total: 1m 50s	remaining: 19.6s
1572:	learn: 0.4562311	total: 1m 50s	remaining: 19.5s
1573:	learn: 0.4562048	total: 1m 51s	remaining: 19.5s
1574:	learn: 0.4561731	total: 1m 51s	remaining: 19.4s
1575:	learn: 0.4561537	total: 1m 51s	remaining: 19.3s
1576:	learn: 0.4561428	total: 1m 51s	remaining: 19.3s
1577:	learn: 0.4561168	total: 1m 51s	remaining: 19.2s
1578:	learn: 0.4560915	total: 1m 51s	remaining: 19.1s
1579:	learn: 0.4560639	total: 1m 51s	remaining: 19s
1580:	learn: 0.4560519	total: 1m 51s	remaining: 19s
1581:	learn: 0.4560271	total: 1m 51s	remaining: 18.9s
1582:	learn: 0.4560269	total: 1m 51s	remaining: 18.8s
1583:	learn: 0.4560041	total: 1m 51s	remaining: 18.8s
1584:	learn: 0.4559873	total: 1m 51s	remaining: 18.7s
1585:	learn: 0.4559704	total: 1m 51s	remaining: 18.6s
1586:	learn: 0.4559473	total: 1m 51s	remaining: 18.6s
1587:	learn: 0.4559293	total: 1m

1723:	learn: 0.4528786	total: 2m 1s	remaining: 8.91s
1724:	learn: 0.4528593	total: 2m 1s	remaining: 8.84s
1725:	learn: 0.4528228	total: 2m 2s	remaining: 8.77s
1726:	learn: 0.4527945	total: 2m 2s	remaining: 8.7s
1727:	learn: 0.4527719	total: 2m 2s	remaining: 8.63s
1728:	learn: 0.4527639	total: 2m 2s	remaining: 8.55s
1729:	learn: 0.4527410	total: 2m 2s	remaining: 8.48s
1730:	learn: 0.4527087	total: 2m 2s	remaining: 8.41s
1731:	learn: 0.4526806	total: 2m 2s	remaining: 8.34s
1732:	learn: 0.4526700	total: 2m 2s	remaining: 8.27s
1733:	learn: 0.4526634	total: 2m 2s	remaining: 8.2s
1734:	learn: 0.4526566	total: 2m 2s	remaining: 8.13s
1735:	learn: 0.4526249	total: 2m 2s	remaining: 8.06s
1736:	learn: 0.4526005	total: 2m 2s	remaining: 7.99s
1737:	learn: 0.4525632	total: 2m 2s	remaining: 7.92s
1738:	learn: 0.4525396	total: 2m 2s	remaining: 7.85s
1739:	learn: 0.4525182	total: 2m 3s	remaining: 7.78s
1740:	learn: 0.4524980	total: 2m 3s	remaining: 7.71s
1741:	learn: 0.4524808	total: 2m 3s	remaining: 7

## <span style="color:purple">Prédiction sur le jeu de test</span>

In [38]:
# Supposez que vous avez de nouvelles données dans une variable `X_new`
# Assurez-vous que les données de `X_new` sont prétraitées de la même manière que les données d'entraînement

# Faire des prédictions individuelles avec chaque modèle de base sur les nouvelles données
lgbm_pred_new = lgb_model.predict_proba(test_officiel)[:, 1]
xgb_pred_new = xgb_model.predict_proba(test_officiel)[:, 1]
catb_pred_new = catboost_model.predict_proba(test_officiel)[:, 1]
gradient_boosting_pred_new = gradient_boosting_model.predict_proba(test_officiel)[:, 1]


# Utiliser les prédictions des modèles de base comme caractéristiques d'entrée pour le modèle de méta-apprentissage
X_blend_new = np.column_stack((lgbm_pred_new, xgb_pred_new, catb_pred_new,gradient_boosting_pred_new))

# Faire une prédiction finale avec le modèle de blending
probabilities = blend_model.predict_proba(X_blend_new)[:, 1]

# Les prédictions finales sont dans `y_pred_new`
df_probabilites = pd.DataFrame(probabilities, columns=['GRAVE'])
df_result_blender_classifier = pd.concat([Num_Acc_test, df_probabilites], axis=1)
df_result_blender_classifier

,Num_Acc,GRAVE
0,201200049538,0.143908
1,201200004221,0.900770
2,201200002457,0.903016
3,201200009834,0.841313
4,201200025602,0.225042
...,...,...
63539,202200008772,0.245883
63540,202200026268,0.119711
63541,202200043480,0.589983
63542,202200022087,0.355640


In [39]:
# Écrire le DataFrame dans un fichier CSV
df_result_blender_classifier.to_csv('blender_classifier_52_.csv', index=False)